In [1]:
# Non-sklearn packages
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

# Sklearn modules & functions
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
# Loading the MNIST dataset
mnist = datasets.fetch_openml('mnist_784', as_frame=True)

/data/2_data_server/cv-07/anaconda3/envs/koreatech/lib/python3.12/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
# Some info about the dataset
print(mnist['DESCR'])

**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [4]:
# Checking the shape of the data
X, y = mnist.data, mnist.target
X.shape, y.shape

((70000, 784), (70000,))

In [5]:
print(type(X))

<class 'pandas.core.frame.DataFrame'>


In [6]:
X, y = np.array(mnist.data), np.array(mnist.target) 
y = y.astype(int)

In [7]:
print(type(X))

<class 'numpy.ndarray'>


In [8]:
len(X[0])

784

In [9]:
config = { 'learning_rate' : 0.01,
            'num_epoch' : 100
          }

In [10]:
X_reshaped = []

for i in range(len(X)):
    X_reshaped.append(X[i].reshape(28, 28))

# X_reshaped는 이제 2차원 이미지 데이터를 포함하는 리스트입니다.
X_reshaped = np.array(X_reshaped)

In [19]:
class Filter:
    def __init__(self, size, num_filters):
        self.size = size
        self.num_filters = num_filters
        self.filters = np.random.randn(num_filters, size, size) / np.sqrt(size * size)
        self.inputs = None
        self.output = None

    def forward(self, inputs):
        self.inputs = inputs
        height, width = inputs.shape
        self.output = np.zeros((height - self.size + 1, width - self.size + 1, self.num_filters))

        for i in range(self.output.shape[0]):
            for j in range(self.output.shape[1]):
                region = inputs[i:i+self.size, j:j+self.size]
                self.output[i, j] = np.sum(region * self.filters, axis=(1, 2))

        return self.output

    def backward(self, output_gradient, learning_rate):
        input_gradient = np.zeros_like(self.inputs)
        filter_gradient = np.zeros_like(self.filters)

        for i in range(self.output.shape[0]):
            for j in range(self.output.shape[1]):
                region = self.inputs[i:i+self.size, j:j+self.size]
                for k in range(self.num_filters):
                    input_gradient[i:i+self.size, j:j+self.size] += output_gradient[i, j, k] * self.filters[k]
                    filter_gradient[k] += output_gradient[i, j, k] * region

        # 필터 업데이트
        self.filters -= learning_rate * filter_gradient

        return input_gradient

In [17]:
class Relu:
    def __init__(self):
        self.input = np.array([])
    def forward(self, input):
        self.input = input
        self.output = np.maximum(0, input)
        return self.output

    def backward(self, output_gradient):
        self.input_gradient = (self.input > 0) * output_gradient
        return self.input_gradient

In [18]:
class cnn:
    def __init__(self, X, y, num_filters):
        self.num_filters = num_filters
        self.X = X
        self.y = y
        self.filter = Filter(3, num_filters)
        self.relu_1 = Relu()
        self.relu_2 = Relu()
        
    
    def conv(self, image):
        pass
    
    def max_pooling(self, image):
        pass

    def forward(self, image):
        pass


In [20]:
class CNN:
    def __init__(self, X, y, num_filters):
        self.num_filters = num_filters
        self.X = X
        self.y = y
        self.filter = Filter(3, num_filters)
        self.relu_1 = Relu()
        self.relu_2 = Relu()
    
    def conv(self, image):
        return self.filter.forward(image)

    def max_pooling(self, image, pool_size=2, stride=2):
        height, width, depth = image.shape
        output_height = (height - pool_size) // stride + 1
        output_width = (width - pool_size) // stride + 1
        pooled = np.zeros((output_height, output_width, depth))

        for i in range(0, height - pool_size + 1, stride):
            for j in range(0, width - pool_size + 1, stride):
                region = image[i:i + pool_size, j:j + pool_size]
                pooled[i // stride, j // stride] = np.max(region, axis=(0, 1))

        return pooled

    def forward(self, image):
        # 첫 번째 합성곱
        conv_output = self.conv(image)
        relu_output = self.relu_1.forward(conv_output)
        
        # 최대 풀링
        pooled_output = self.max_pooling(relu_output)
        
        # 두 번째 ReLU
        relu_output_2 = self.relu_2.forward(pooled_output)

        return relu_output_2

In [33]:
cnn_model = CNN(X[0:100], y[0:100], num_filters=8)
output = cnn_model.forward(X[0:100])
print(output.shape)
# for i in range(len(output[0])):
#     if(output[i] != 0):
#         print("Output:\n", output[i])
#         break
print("Final output:\n", output)

(49, 391, 8)
Final output:
 [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0

In [32]:
cnn_model = CNN(X[0:10], y[0:10], num_filters=8)
output = cnn_model.forward(X[0:10])
print(output.shape)
# for i in range(len(output[0])):
#     if(output[i] != 0):
#         print("Output:\n", output[i])
#         break
print("Final output:\n", output)

(4, 391, 8)
Final output:
 [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
